In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

In [ ]:
import statsmodels.api as sm
import statsmodels.discrete.discrete_model as dm
from patsy import dmatrices
import statsmodels.graphics.tsaplots as tsa
from statsmodels.base.model import GenericLikelihoodModel
from scipy.stats import poisson
from scipy.stats import binom
from statsmodels.tsa.deterministic import TimeTrend, Seasonality

In [ ]:
# data
# https://data.cityofnewyork.us/Transportation/Bicycle-Counts-for-East-River-Bridges-Historical-/gua4-p9wg

In [ ]:
bikes = pd.read_csv('bikes.csv', header=0, index_col='Date',)

In [ ]:
bikes.index = pd.to_datetime(bikes.index,)

In [ ]:
pd.infer_freq(bikes.index)

In [ ]:
bikes.shape

In [ ]:
bikes = np.floor(bikes/100).astype(int)

In [ ]:
bikes.tail()

In [ ]:
ax = bikes.Manhattan_Bridge.plot(figsize=(9,6), lw=1, color='r' )
plt.title('Daily bikes Manhattan Bridge')

In [ ]:

train_len = 145
h = 13
bikes_train = bikes.iloc[:train_len]
 
bikes_test = bikes.iloc[train_len:(train_len+h)]
bikes_test.shape

In [ ]:
bikes_train.tail()

In [ ]:
bikes_test.head()

In [ ]:
pd.Timedelta(1,unit='D')

# Poisson autogregression

In [ ]:
def PoissonAutoRegForecast(endog, p=1, h=1, exog=None, add_one=True):
    '''
    Code development based on this post: 
    https://timeseriesreasoning.com/contents/poisson-regression-models-for-time-series-data/
    
    Parameters
    ----------
    endog: pd.Series
        The endogenous time series to forecast.
    p: int, optional, default =1
        The order of the Autoregression model.
    h: int, optional, default =1
        The forecast horizon.
    exog: pd.DataFrame or pd.Series, optional, default =None
        The exogenous covariates, if any. Note in thsi function you must use exog
        argument to feed in the training set and the predicton set in one Series/DataFrame.
    add_one: bool, optional, default =True
        Whether to add one to all counts to avoid taking the log of zeros.
    '''
    
    ## create the log lagged values
    log_lag_endog = pd.concat([np.log(endog+int(add_one)).shift(a) for a in range(1,p+1)], axis=1).dropna()
    log_lag_endog.columns = ['log_lag'+str(a) for a in range(1,p+1)]
    # concat with other exog variables
    if exog is not None:
        exog_with_lags = pd.concat([log_lag_endog, exog], axis=1).dropna()
    else:
        exog_with_lags = log_lag_endog
    
    # add an intercept
    exog_with_lags.insert(loc=0, column= 'intercept', value=1)
    # trim the data to account for taking the lags
    endog = endog.iloc[endog.index.isin(exog_with_lags.index)]
    # fit the Poissin GLM
    poisson_model = dm.Poisson(endog=endog, exog=exog_with_lags).fit(maxiter=300)
    
    # recursive forecast
    for c in range(1,h+1):
        # find the next time steps in the exog df to use as predictors
        if exog is not None:
            exog_predict = np.array((1, *np.log(endog[-p:]),
                            *exog.loc[endog.index[-1]+pd.Timedelta(1,pd.infer_freq(endog.index))] ) )
        else:
            exog_predict = np.array((1, *np.log(endog[-p:] )) )
            
        # multiply the model parameters by the covariates for the next time step    
        pred = np.exp(
               np.sum(
                   poisson_model.params * exog_predict
                   ) )
        endog.loc[ endog.index[-1] + pd.Timedelta(1, pd.infer_freq(endog.index))] = pred
    
    # return the forecast
    return endog[-h:]
    
    

In [ ]:
trend_gen = TimeTrend(constant=False, order=2)
trend_df = trend_gen.in_sample(bikes.index)
seas_gen = Seasonality(period=7)
seas_df = seas_gen.in_sample(bikes.index)
#test_df = trend_gen.in_sample(bikes_test.index)
seas_df.drop(columns='s(1,7)', inplace=True)

In [ ]:
p=2
add_one=True
endog = bikes_train.Brooklyn_Bridge
log_lag_endog = pd.concat([np.log(endog+int(add_one)).shift(a) for a in range(1,p+1)], axis=1).dropna()
log_lag_endog.columns = ['log_lag'+str(a) for a in range(1,p+1)]
log_lag_endog.insert(loc=0, column='intercept', value=1)
endog = endog.iloc[endog.index.isin(log_lag_endog.index)]

In [ ]:
exog_with_lags = pd.concat([log_lag_endog, trend_df], axis=1).dropna()

In [ ]:
exog_with_lags.tail()

In [ ]:
sm.GLM(endog, exog=exog_with_lags, family=sm.families.Poisson()).fit().summary()

In [ ]:
poisson_model = dm.Poisson(endog=endog, exog=exog_with_lags).fit(maxiter=300)

In [ ]:
poisson_model.summary()

In [ ]:
poisson_model.params

In [ ]:
pd.infer_freq(endog.index)

In [ ]:
endog.index[-1]+pd.Timedelta(1,'D')

In [ ]:
trend_df.loc[endog.index[-1]+pd.Timedelta(1,'D')]

In [ ]:
poisson_model = dm.Poisson(endog=endog, exog=log_lag_endog).fit(maxiter=100)

In [ ]:
np.exp(poisson_model.params[0] + poisson_model.params[1]*np.log(bikes_train.Brooklyn_Bridge[-1]) )

In [ ]:
np.sum(poisson_model.params * np.array((1, *np.log(bikes_train.Brooklyn_Bridge[-2]), 
                                        *trend_df.loc[endog.index[-1]+pd.Timedelta(1,'D')][:] ) ) )

In [ ]:
poisson_model.params*np.array( (1, *np.log(bikes_train.Brooklyn_Bridge[-2:]),
           *trend_df.loc[endog.index[-1]+pd.Timedelta(1,'D')] ) )

In [ ]:
trend_df.loc[endog.index[-1]+pd.Timedelta(1,'D')][:]

In [ ]:
np.log(bikes_train.Brooklyn_Bridge[-2:])[:]

In [ ]:
pois_cast = PoissonAutoRegForecast(bikes_train.Manhattan_Bridge, p=5, h=14, exog=seas_df)

In [ ]:
ax = bikes_train.Manhattan_Bridge.plot(figsize=(9,7))
pois_cast.plot(ax=ax, color='r')
bikes_test.Manhattan_Bridge.plot(ax=ax, color='k')

In [ ]:
pois_cast = PoissonAutoRegForecast(bikes_train.Queensboro_Bridge, p=5, h=14, exog=seas_df)

In [ ]:
ax = bikes_train.Queensboro_Bridge.plot(figsize=(9,7))
pois_cast.plot(ax=ax, color='r')
bikes_test.Queensboro_Bridge.plot(ax=ax, color='k')

# Poisson INAR(p)
https://timeseriesreasoning.com/contents/the-poisson-inar1-regression-model/

In [ ]:
# strikes dataset
strikes_dataset = sm.datasets.get_rdataset(dataname='StrikeNb', package='Ecdat')


In [ ]:
strikes_data = strikes_dataset.data.copy()
strikes_data_train = strikes_data.query('time<=92')
strikes_data_test = strikes_data.query('time>92').reset_index().drop('index', axis=1)
strikes_data.head()

In [ ]:
expr = 'strikes ~ output'
y_train, X_train = dmatrices(expr, strikes_data_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, strikes_data_test, return_type='dataframe')

In [ ]:
class INAR(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(INAR, self).__init__(endog, exog, **kwds)

In [ ]:
class PoissonINAR(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(PoissonINAR, self).__init__(endog, exog, **kwds)
        
    def nloglikeobs(self, params):
        #Fetch the parameters gamma and beta 
        #that we would be optimizing
        gamma = params[-1]
        beta = params[:-1]
         
        #Set y and X
        y = self.endog
        y = np.array(y)
        X = self.exog
         
        #Compute rho as a function of gamma
        rho = 1.0/(1.0+math.exp(-gamma))
         
        #Compute the Poisson mean mu as the exponentiated dot 
        #product of X and Beta
        mu = np.exp(X.dot(beta))
        #Init the list of log-likelihhod values, 
        #one value for each y
        ll = []
        
        #Compute all the log-likelihood values for 
        #the Poisson INAR(1) model
        for t in range(len(y)-1,0,-1):
            prob_y_t = 0
            for j in range(int(min(y[t], y[t-1])+1)):
                prob_y_t += poisson.pmf((y[t]-j), mu[t]) * binom.pmf(j, y[t-1], rho)
            ll.append(np.log(prob_y_t))
        ll = np.array(ll)
        #return the negated array of log-likelihoods
        return -ll
    #Let’s also implement the model.fit() method:

    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
    #Add the gamma parameter to the list of 
    #exogneous variables that the model will optimize
        self.exog_names.append('gamma')
        
        if start_params == None:
            #Start with some initial values of Beta and gamma
            start_params = np.append(np.ones(self.exog.shape[1]), 1.0)
            
        #Call super.fit() to start the training
            return super(PoissonINAR, self).fit(start_params=start_params,
                maxiter=maxiter, maxfun=maxfun, **kwds)
        
    def predict(self, params, exog=None, *args, **kwargs):
        #Fetch the optimized values of parameters gamma and beta
        fitted_gamma = params[-1]
        fitted_beta = params[:-1]
        
        #Compute rho as a function of gamma
        rho = 1.0/(1.0+math.exp(-fitted_gamma))
        
        #Get the Intercept and the regression variables,
        #Don't get the last column which contains the lagged y values
        X = exog[:,:-1]
        #Fetch the lagged y values
        y_lag_1 = exog[:,-1]
        
        #Compute the predicted y using the fitted Poisson INAR(1) 
        #model's equation
        y_pred = rho * y_lag_1 + np.exp(X.dot(fitted_beta))
        
        return y_pred

In [ ]:
inar_model = PoissonINAR(y_train, X_train)
inar_model_results = inar_model.fit()

In [ ]:
inar_model_results.summary()